# --- Day 6: Guard Gallivant ---
The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single guard is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:
```
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
```
The map shows the current position of the guard with ^ (to indicate the guard is currently facing up from the perspective of the map). Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

If there is something directly in front of you, turn right 90 degrees.
Otherwise, take a step forward.
Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):

```
....#.....
....^....#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```
Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:
```
....#.....
........>#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```
Reaching another obstacle (a spool of several very long polymers), she turns right again and continues downward:
```
....#.....
.........#
..........
..#.......
.......#..
..........
.#......v.
........#.
#.........
......#...
```
This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):
```
....#.....
.........#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#v..
```
By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. Including the guard's starting position, the positions visited by the guard before leaving the area are marked with an X:
```
....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
```
In this example, the guard will visit 41 distinct positions on your map.

Predict the path of the guard. How many distinct positions will the guard visit before leaving the mapped area?



In [1]:
import requests

# ---------------------------------------------------------
# FETCH INPUT FROM ADVENT OF CODE
# ---------------------------------------------------------
SESSION = "53616c7465645f5f988f586cbd86c3110501a5d7843d1cf8606ac44f4daa4f68d8b1daa6906dc9dbb21c0c1b70057d32c31af4590ce27e9dc01dcff65b01c12c"
  # session cookie 

def fetch_input(year, day):
    url = f"https://adventofcode.com/{year}/day/{day}/input"
    cookies = {"session": SESSION}
    r = requests.get(url, cookies=cookies)
    r.raise_for_status()
    return r.text.strip().splitlines()


# ---------------------------------------------------------
# SIMULATE THE GUARD
# ---------------------------------------------------------
# ^ = up, > = right, v = down, < = left
DIRS = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1),
}
TURN_RIGHT = {
    "^": ">",
    ">": "v",
    "v": "<",
    "<": "^",
}

def simulate(grid):
    R, C = len(grid), len(grid[0])

    # find guard's starting position + direction
    for r in range(R):
        for c in range(C):
            if grid[r][c] in "^>v<":
                guard_r, guard_c = r, c
                direction = grid[r][c]
                break

    visited = set()
    visited.add((guard_r, guard_c))

    while True:
        dr, dc = DIRS[direction]
        nr, nc = guard_r + dr, guard_c + dc

        # guard leaves the grid → stop
        if not (0 <= nr < R and 0 <= nc < C):
            break

        # obstacle → turn right
        if grid[nr][nc] == "#":
            direction = TURN_RIGHT[direction]
            continue

        # valid move → step forward
        guard_r, guard_c = nr, nc
        visited.add((guard_r, guard_c))

    return len(visited)


# ---------------------------------------------------------
# RUN EVERYTHING
# ---------------------------------------------------------
if __name__ == "__main__":
    grid = fetch_input(2024, 6)
    result = simulate(grid)
    print("Part 1 — Distinct positions visited:", result)


Part 1 — Distinct positions visited: 5531


# What the script does
1. Loads your input directly from the AoC 2024 Day 6 page with your session cookie.
2. Finds the guard (^, >, v, or <) in the grid.
3. Repeatedly applies the rules:
```
If the tile in front is a #, 
    turn right
Else
    step forward
Continue until the guard steps outside the map
````
4. Tracks all distinct positions visited including the starting square.
5. Prints the final answer.

# Part 1 and Part 2 combined

### 🟦 Part 1
Runs the guard simulation normally → counts the tiles visited before exiting the map.
### 🟥 Part 2
1. Generates the guard’s full path from Part 1.
2. For each visited tile:
3. Places a temporary # obstacle there
4. Re-simulates
5. Detects infinite loops by checking repeated (r, c, direction)
6. Counts how many positions cause a loop.

In [2]:
import requests

# ---------------------------------------------------------
# FETCH INPUT FROM AOC
# ---------------------------------------------------------
SESSION = "53616c7465645f5f988f586cbd86c3110501a5d7843d1cf8606ac44f4daa4f68d8b1daa6906dc9dbb21c0c1b70057d32c31af4590ce27e9dc01dcff65b01c12c"
  # session cookie 
    
def fetch_input(year, day):
    url = f"https://adventofcode.com/{year}/day/{day}/input"
    cookies = {"session": SESSION}
    r = requests.get(url, cookies=cookies)
    r.raise_for_status()
    return r.text.strip().splitlines()


# ---------------------------------------------------------
# DIRECTIONS
# ---------------------------------------------------------
DIRS = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1),
}
TURN_RIGHT = {
    "^": ">",
    ">": "v",
    "v": "<",
    "<": "^",
}

# ---------------------------------------------------------
# SIMULATE UNTIL EXIT (RETURN VISITED COUNT)
# ---------------------------------------------------------
def simulate_until_exit(grid, start_r, start_c, start_dir):
    R, C = len(grid), len(grid[0])
    visited = set()
    r, c, d = start_r, start_c, start_dir
    visited.add((r, c))

    while True:
        dr, dc = DIRS[d]
        nr, nc = r + dr, c + dc

        if not (0 <= nr < R and 0 <= nc < C):
            return visited  # exits map → stop

        if grid[nr][nc] == "#":
            d = TURN_RIGHT[d]
        else:
            r, c = nr, nc
            visited.add((r, c))


# ---------------------------------------------------------
# SIMULATE AND CHECK IF LOOP OCCURS (PART 2)
# ---------------------------------------------------------
def causes_loop(grid, start_r, start_c, start_dir):
    """
    Returns True if this modified grid causes the guard to loop forever.
    Loop detection: (r, c, direction) repeating means infinite loop.
    """
    R, C = len(grid), len(grid[0])
    r, c, d = start_r, start_c, start_dir

    seen_states = set()
    seen_states.add((r, c, d))

    while True:
        dr, dc = DIRS[d]
        nr, nc = r + dr, c + dc

        # guard exits → NOT a loop
        if not (0 <= nr < R and 0 <= nc < C):
            return False

        # obstacle → turn right
        if grid[nr][nc] == "#":
            d = TURN_RIGHT[d]
        else:
            r, c = nr, nc

        # loop if same (position, direction) reappears
        state = (r, c, d)
        if state in seen_states:
            return True
        seen_states.add(state)


# ---------------------------------------------------------
# PART 2: TRY ADDING ONE OBSTACLE
# ---------------------------------------------------------
def count_loop_positions(original_grid, start_r, start_c, start_dir):
    R, C = len(original_grid), len(original_grid[0])

    # First get all positions guard visits in the unmodified grid
    baseline_path = simulate_until_exit(original_grid, start_r, start_c, start_dir)

    loop_count = 0

    for (rr, cc) in baseline_path:
        # Don't place obstacle on the starting square
        if (rr, cc) == (start_r, start_c):
            continue

        # skip existing obstacles
        if original_grid[rr][cc] == "#":
            continue

        # create a modified grid with an obstacle inserted
        new_grid = [list(row) for row in original_grid]
        new_grid[rr][cc] = "#"

        if causes_loop(new_grid, start_r, start_c, start_dir):
            loop_count += 1

    return loop_count


# ---------------------------------------------------------
# MAIN
# ---------------------------------------------------------
if __name__ == "__main__":
    grid = fetch_input(2024, 6)
    R, C = len(grid), len(grid[0])

    # find starting guard
    for r in range(R):
        for c in range(C):
            if grid[r][c] in "^>v<":
                start_r, start_c = r, c
                start_dir = grid[r][c]
                break

    # PART 1
    visited_positions = simulate_until_exit(grid, start_r, start_c, start_dir)
    print("Part 1 — Distinct visited positions:", len(visited_positions))

    # PART 2
    loops = count_loop_positions(grid, start_r, start_c, start_dir)
    print("Part 2 — Loop-causing obstruction positions:", loops)


Part 1 — Distinct visited positions: 5531
Part 2 — Loop-causing obstruction positions: 2165
